In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
from torchvision import datasets
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.models as models

In [18]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('device:', device)

device: cpu


In [24]:
!mv kaggle.json /root/.kaggle

In [25]:
!kaggle datasets download -d gpiosenka/sports-classification

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 403, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [ ]:
!unzip sports-classification.zip >> /dev/null

unzip:  cannot find or open sports-classification.zip, sports-classification.zip.zip or sports-classification.zip.ZIP.


In [ ]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize images to the desired size
    transforms.ToTensor(),           # Convert images to tensors
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize images
])

In [ ]:
# train data loader and test data loader
dataset = ImageFolder(root='./train', transform=transform)

batch_size = 128
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

test_dataset = ImageFolder(root='./test', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

FileNotFoundError: [Errno 2] No such file or directory: './train'

In [ ]:
def train_and_return_test_accuracy(model, num_epochs=20, learning_rate=0.005):
    """Function to train a model on the training set and evaluate its accuracy on the test set"""
    # Setup for training: using stochastic gradient descent for optimization
    sgd_optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
    criterion = nn.CrossEntropyLoss()

    # Iterative training process
    for epoch in range(num_epochs):
        for batch_index, (inputs, labels) in enumerate(train_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            sgd_optimizer.zero_grad()
            predictions = model(inputs)
            loss_value = criterion(predictions, labels)
            loss_value.backward()
            sgd_optimizer.step()
        if (epoch + 1) % 5 == 0:
            print(f'Training Epoch: {epoch + 1} \tLoss: {loss_value.item():.6f}')

    # Evaluating accuracy on the test dataset
    correct_predictions = 0
    all_samples = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            test_outputs = model(inputs)
            _, preds = torch.max(test_outputs.data, 1)
            all_samples += 1
            correct_predictions += (preds == labels).sum().item()
    accuracy_on_test = correct_predictions / all_samples
    return accuracy_on_test

In [ ]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=False)
# modify the last layer to fit the number of classes 100
model.classifier[6] = nn.Linear(4096, 100)
model = model.to(device)

In [ ]:
test_accuracy = train_and_return_test_accuracy(model)
print(f'Test accuracy: {test_accuracy} % ')

### 2. Transfer learning


In [ ]:
def train_and_return_test_accuracy(model, total_epochs=20, step_lr=0.005):
    """Function for training a model on the training dataset and calculating its accuracy on a test dataset"""
    # Setup for optimization with stochastic gradient descent
    optimize = optim.SGD(model.parameters(), lr=step_lr, momentum=0.9)
    calculate_loss = nn.CrossEntropyLoss()

    # Training loop
    for current_epoch in range(total_epochs):
        for batch_number, (input_data, labels) in enumerate(train_loader):
            input_data = input_data.to(device)
            labels = labels.to(device)
            optimize.zero_grad()
            prediction = model(input_data)
            training_loss = calculate_loss(prediction, labels)
            training_loss.backward()
            optimize.step()
        if (current_epoch + 1) % 5 == 0:
            print(f'Training Epoch: {current_epoch + 1} \tLoss: {training_loss.item():.6f}')

    # Evaluation for accuracy on the test set
    accurate_predictions = 0
    total_predictions = 0
    with torch.no_grad():
        for input_data, labels in test_loader:
            input_data = input_data.to(device)
            labels = labels.to(device)
            test_predictions = model(input_data)
            _, highest_predictions = torch.max(test_predictions.data, 1)
            total_predictions += 1
            accurate_predictions += (highest_predictions == labels).sum().item()
    accuracy_of_test = accurate_predictions / total_predictions
    return accuracy_of_test


In [ ]:
# Transfer learning

model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)
# modify the last layer to fit the number of classes 100
model.classifier[6] = nn.Linear(4096, 100)

# freeze all feature layers
for param in model.features.parameters():
    param.requires_grad = False

model = model.to(device)

test_accuracy = train_and_return_test_accuracy(model)
print(f'Test accuracy: {test_accuracy}')

### 3. Transfer learning based on vgg19 and resnet50

In [ ]:
def train_and_return_test_accuracy(model, total_epochs=5, step_lr=0.005):
    """Function for training a model on the training dataset and calculating its accuracy on a test dataset"""
    # Setup for optimization with stochastic gradient descent
    optimize = optim.SGD(model.parameters(), lr=step_lr, momentum=0.9)
    calculate_loss = nn.CrossEntropyLoss()

    # Training loop
    for current_epoch in range(total_epochs):
        for batch_number, (input_data, labels) in enumerate(train_loader):
            input_data = input_data.to(device)
            labels = labels.to(device)
            optimize.zero_grad()
            prediction = model(input_data)
            training_loss = calculate_loss(prediction, labels)
            training_loss.backward()
            optimize.step()
        if (current_epoch + 1) % 1 == 0:
            print(f'Training Epoch: {current_epoch + 1} \tLoss: {training_loss.item():.6f}')

    # Evaluation for accuracy on the test set
    accurate_predictions = 0
    total_predictions = 0
    with torch.no_grad():
        for input_data, labels in test_loader:
            input_data = input_data.to(device)
            labels = labels.to(device)
            test_predictions = model(input_data)
            _, highest_predictions = torch.max(test_predictions.data, 1)
            total_predictions += 1
            accurate_predictions += (highest_predictions == labels).sum().item()
    accuracy_of_test = accurate_predictions / total_predictions
    return accuracy_of_test

In [ ]:
vgg19 = models.vgg19(pretrained=True)
vgg19.classifier[6] = nn.Linear(4096, 100)

# freeze all feature layers
for param in vgg19.features.parameters():
    param.requires_grad = False

In [ ]:
vgg19 = vgg19.to(device)

test_accuracy = train_and_return_test_accuracy(vgg19)
print(f'Test accuracy: {test_accuracy}')

In [ ]:
def train_and_return_test_accuracy(model, total_epochs=5, step_lr=0.005):
    """Function for training a model on the training dataset and calculating its accuracy on a test dataset"""
    # Setup for optimization with stochastic gradient descent
    optimize = optim.SGD(model.parameters(), lr=step_lr, momentum=0.9)
    calculate_loss = nn.CrossEntropyLoss()

    # Training loop
    for current_epoch in range(total_epochs):
        for batch_number, (input_data, labels) in enumerate(train_loader):
            input_data = input_data.to(device)
            labels = labels.to(device)
            optimize.zero_grad()
            prediction = model(input_data)
            training_loss = calculate_loss(prediction, labels)
            training_loss.backward()
            optimize.step()
        if (current_epoch + 1) % 1 == 0:
            print(f'Training Epoch: {current_epoch + 1} \tLoss: {training_loss.item():.6f}')

    # Evaluation for accuracy on the test set
    accurate_predictions = 0
    total_predictions = 0
    with torch.no_grad():
        for input_data, labels in test_loader:
            input_data = input_data.to(device)
            labels = labels.to(device)
            test_predictions = model(input_data)
            _, highest_predictions = torch.max(test_predictions.data, 1)
            total_predictions += 1
            accurate_predictions += (highest_predictions == labels).sum().item()
    accuracy_of_test = accurate_predictions / total_predictions
    return accuracy_of_test

In [ ]:
resnet50 = models.resnet50(pretrained=True)
resnet50.classifier[6] = nn.Linear(4096, 100)

# freeze all feature layers
for param in resnet50.features.parameters():
    param.requires_grad = False

In [ ]:
resnet50 = resnet50.to(device)

test_accuracy = train_and_return_test_accuracy(resnet50)
print(f'Test accuracy: {test_accuracy}')